In [1]:
import can
from joint_can import *
import numpy as np
import os
import time
import math

In [2]:
def initialize_joint(bus):
    send_msg(0x0A0, None, bus)
    time.sleep(1)
    
    send_msg(0x0b0, [0,0,0,0,0,0], bus)
    time.sleep(0.1)
    send_msg(0x040, None, bus)
    time.sleep(0.1)
    send_msg(0x0a0, None, bus)
    time.sleep(1)
    
    if (send_msg(0x0F0, [1, 2], bus).data[0] == 1):
        send_msg(0x010, [2], bus)
        time.sleep(1)
        send_msg(0x010, [3], bus).data[0]
        time.sleep(5)
        return True
    else:
        return False
     

In [3]:
diag_samples = 300
const_torque = 8
# serial_number
# note
run_duration = 2

In [85]:
const_torque=8. #zadawany moment obrotowy (konieczna kropka na końcu jeżeli jest to liczba całkowita) 16 dla J25, 8 dla J20
serial_nr="JANUSZ_3_RI70_21414876" #musi byc min jedna litera w numerze
note="testy arka" #póki co musi być cokolwiek
file_path = "/home/ubuntu/joint_diagnostic/diagnostic_data"

In [92]:
def save_data(data: np.array, file_path: str, note: str, serial_nr: str, torque: float, temps: list) -> None:
    with open(file_path+'/'+serial_nr+'/'+serial_nr+time.strftime('_%y_%m_%d_%H_%M')+'.txt','w') as file:
        file.write("serial_number: " + serial_nr+'\n')
        file.write("torque: "+str(torque)+'\n')
        
        file.write("start_temps: "+str(temps[1][0]) +" ("+str(temps[0][0])+")\n")
        file.write("finish_temps: "+str(temps[1][1]) +" ("+str(temps[0][1])+")\n")
        
        file.write("note: " + note+'\n')
        for line in data:
            file.write(f'{line[0]:.6f} {line[1]:.6f}\n')

In [5]:
os.popen('/home/ubuntu/can_down.sh')
time.sleep(1)
os.popen('/home/ubuntu/can_up.sh')
time.sleep(1)

bus = can.interface.Bus(bustype='socketcan', channel='can0', bitrate=1000000, data_bitrate=5000000, fd=True)
while not initialize_joint(bus):
    print("Joint initialization in progress")

In [6]:
# process flow
# check_connection
# joint_init
# param_init
# var_init
# joint_position_init
# diagnostic_loop
# save_data

In [8]:
print("Running diagnostics...")
position_bins=np.linspace(0,2*math.pi,diag_samples)
start_time=time.time()
data=[]

motor_temp=[0,0]
bearing_temp=[0,0]

current_reading = set_torque(const_torque,bus)
motor_temp[0] = current_reading.motor_temp
bearing_temp[0] = current_reading.bearing_temp

time.sleep(5)
while((cur_time:=time.time()-start_time)<(run_duration*60)):
    current_reading=set_torque(const_torque,bus)
    print(f'\r Time left: {(math.floor(run_duration-cur_time/60))} min {int((run_duration*60-cur_time)%60)} sec\t\t', end='')
    if np.sign(current_reading.torque) == np.sign(const_torque):
        motor_temp[1] = current_reading.motor_temp
        bearing_temp[1] = current_reading.bearing_temp
        data.append([current_reading.position + math.pi, current_reading.speed])
    else:
        if cur_time>30:
            print("Speed/Torque sign error, aborting.")
            break
    
        
set_torque(0.,bus)

Running diagnostics...
 Time left: 0 min 0 sec			

In [73]:
data = np.array(data)
position_bin_num = np.digitize(data[:,0],position_bins)
binarized_avg_data = np.array([position_bins,np.zeros(diag_samples)]).T
for i in range(diag_samples):
    binarized_avg_data[i,1]=np.mean(data[position_bin_num==i,1])

In [93]:
save_data(binarized_avg_data, path, note, serial_nr, const_torque, [motor_temp, bearing_temp])